In [1]:
import pandas as pd

In [2]:
from datasets import load_dataset

/root/anaconda3/envs/research_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Download first 20000 examples from the dataset (assuming it supports slicing)
dataset = load_dataset("m-aliabbas1/myeidi")

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
# !pip install -U datasets 

In [5]:
# !ls

In [6]:
# !rm -rf ~/.cache/huggingface/datasets/

In [7]:
dataset = dataset["train"].train_test_split(seed=42)["train"]

In [8]:
dataset

Dataset({
    features: ['text', 'audio', 'duration'],
    num_rows: 78390
})

In [14]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [15]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'audio', 'duration'],
        num_rows: 62712
    })
    test: Dataset({
        features: ['text', 'audio', 'duration'],
        num_rows: 15678
    })
})

In [16]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [17]:
train_dataset

Dataset({
    features: ['text', 'audio', 'duration'],
    num_rows: 62712
})

In [18]:
import csv
import os



In [19]:
# Prepare metadata lines
metadata = [["audio_file", "text", "speaker_name"]]



In [20]:
# df = pd.read_csv('metadata_train.csv')

In [15]:
!ls wavs

ls: cannot access 'wavs': No such file or directory


In [18]:
# !pip install librosa

In [20]:
# !pip install librosa soundfile tqdm


In [21]:
import os
import csv
import librosa
import soundfile as sf
from tqdm import tqdm

def process_dataset_to_metadata(
    dataset, 
    output_dir="wavs", 
    metadata_path="metadata_train.csv", 
    target_sr=22500, 
    speaker_name="@faisal"
):
    """
    Resamples and saves audio files from Hugging Face dataset and creates metadata CSV.
    
    Args:
        dataset: Hugging Face dataset with 'audio' and 'text' fields
        output_dir: Directory to save resampled audio files
        metadata_path: Path to save metadata CSV file
        target_sr: Target sample rate (default is 24000 Hz)
        speaker_name: Speaker name to use in metadata

    Returns:
        None (saves audio files and CSV to disk)
    """
    os.makedirs(output_dir, exist_ok=True)
    metadata = [["audio_file", "text", "speaker_name"]]

    for example in tqdm(dataset, desc="Processing audio"):
        audio_data = example["audio"]
        text = example["text"]
        filename = os.path.basename(audio_data["path"])
        new_filename = filename.replace(".wav", f"_{target_sr//1000}k.wav")
        output_path = os.path.join(output_dir, new_filename)

        # Resample and save
        y = audio_data["array"]
        sr = audio_data["sampling_rate"]
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
        sf.write(output_path, y_resampled, target_sr)

        # Append metadata row
        metadata.append([output_path, text, speaker_name])

    # Write metadata to CSV
    with open(metadata_path, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f, delimiter='|')
        writer.writerows(metadata)

    print(f"✅ Done: {len(dataset)} samples processed.")
    print(f"📁 Audio saved in: {output_dir}")
    print(f"📝 Metadata saved to: {metadata_path}")


In [22]:
# !conda install numpy=2.1 -y


In [23]:
process_dataset_to_metadata(train_dataset)

Processing audio: 100%|█████████████████████████████████████████████████████████████| 62712/62712 [01:04<00:00, 967.97it/s]


✅ Done: 62712 samples processed.
📁 Audio saved in: wavs
📝 Metadata saved to: metadata_train.csv


In [24]:
# !rm -rf wavs

In [25]:
test_dataset = split_dataset["test"]

In [26]:
process_dataset_to_metadata(test_dataset,output_dir="wavs", 
    metadata_path="metadata_test.csv", 
    target_sr=24000, 
    speaker_name="@faisal")

Processing audio: 100%|█████████████████████████████████████████████████████████████| 15678/15678 [00:27<00:00, 570.39it/s]


✅ Done: 15678 samples processed.
📁 Audio saved in: wavs
📝 Metadata saved to: metadata_test.csv


In [37]:
!mv wavs ./XTTSv2-Finetuning-for-New-Languages/datasets-1/